In [1]:
import os
os.environ['TZ'] = 'GMT'
os.environ["OMP_NUM_THREADS"] = "4"
import h5py
import emcee
import pickle
import datetime
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["font.size"] = 12

from tqdm import tqdm
from MCMC_func import *
from multiprocessing import Pool, cpu_count

ncpu = cpu_count()
print("{0} CPUs".format(ncpu))
from das_util import read_decimate, get_tstamp

112 CPUs


In [2]:
output_imgdir = "./figure/MCMC_modelfit"
output_imgdir_debug = "./figure/MCMC_modelfit_dvvtrace"
output_datadir = "./processed_data/MCMC_sampler/no_cc"

# set random seed to fix the initial chains
np.random.seed(seed=105)
if not os.path.exists(output_imgdir):
    os.makedirs(output_imgdir)
if not os.path.exists(output_imgdir_debug):
    os.makedirs(output_imgdir_debug)
if not os.path.exists(output_datadir):
    os.makedirs(output_datadir)

In [3]:
with h5py.File('/fd1/QibinShi_data/England_farm/interp_dvv_temp_moisture_dts.hdf5', 'r') as f:
    interp_cc = f['cc'][:]
    interp_temp10 = f['temperature10'][:]
    interp_temp0 = f['temperature0'][:]
    interp_humidity = f['moisture'][:]
    ave_psd = f['psd_rain'][:]
    
with h5py.File('/fd1/QibinShi_data/England_farm/interp_dvv_temp_moisture.hdf5', 'r') as f:
    interp_dvv = f['dvv_refined'][:]

with h5py.File('/fd1/QibinShi_data/England_farm/psd_all.hdf5', 'r') as f:
    file_list = f['daslist'][:].astype('U')
#     PSD_all_time = f['psd_all_time'][:]
#     freq = f['frequency'][:]

acqu_time = np.array([get_tstamp(i) for i in file_list])
unix_tvec = np.array([x.timestamp for x in acqu_time])

In [4]:
# format is: (initial value, [vmin, vmax])
modelparam = {
            "a0"            : (0.0, [-5.0, 5.0]), # offset
            "p1"            : (-0.05, [-10, 0.0]), # scale of GWL 
            "a_{precip}"      : (1e-2, [0, 0.05]), # delay in GWL [1/hour]
            "p2"            : (1, [0, 20]), # scale of Temp
            "t_{shiftdays}"   : (0.1, [0, 100]), # shift of temp in 1 sec
            "p3"            : (1, [0, 20]), # scale of Temp
            "log_f"         : (5, [-10, 10]), # magnitude of uncertainity
            }

modelparam["fixparam01"] = False
modelparam["nsteps"] = 5000
modelparam["nwalkers"] = 32

modelparam["precip"] = ave_psd * 3.5
modelparam["CAVG"] = (interp_temp10-7.5)/20
modelparam["a_{precip}_fixed"] = 0.02426

modelparam["averagestack_step"] = 1
modelparam["uniform_tvec"] = acqu_time
modelparam["unix_tvec"] = unix_tvec

In [ ]:
channel_ids = np.arange(51)

for modelcase in ["evap"]:
    modelparam["modelcase"] = modelcase
    # Initialize Markov Chain
    pos, ndim, keys = get_init_param(**modelparam)
    modelparam["keys"] = keys

    if modelparam["fixparam01"]:
        pos_fixed = np.delete(pos, [keys.index("a_{precip}")], 1)
        modelparam["pos"] = pos_fixed
        modelparam["ndim"] = ndim - 1
    else:
        modelparam["pos"] = pos
        modelparam["ndim"] = ndim

    for channel_id in channel_ids:

        print("start processing Ch."+ str(channel_id))
        modelparam["dvv_data_trim"] = interp_dvv[channel_id] - np.mean(interp_dvv[channel_id])
        modelparam["err_data_trim"] = (1-interp_cc[channel_id])*2
        modelparam["surface_temp"] = (interp_temp0[channel_id]-5)/60

        with Pool() as pool:
            sampler = emcee.EnsembleSampler(
                            modelparam["nwalkers"], modelparam["ndim"], log_probability,
                            moves=emcee.moves.StretchMove(),
                            kwargs=(modelparam),
                            pool=pool)
            sampler.run_mcmc(modelparam["pos"], modelparam["nsteps"], blobs0=True, progress=True)

        # Save the current state.
        foname = output_datadir + "/MCMC_sampler_Ch%s_%s_%s.pickle" % (
        str(channel_id), "25_50Hz", modelparam["modelcase"])
        with open(foname, "wb") as f:
            pickle.dump(sampler, f)
            pickle.dump(modelparam, f)


start processing Ch.0


100%|██████████| 5000/5000 [06:29<00:00, 12.84it/s]


start processing Ch.1


100%|██████████| 5000/5000 [06:27<00:00, 12.89it/s]


start processing Ch.2


...

100%|██████████| 5000/5000 [06:36<00:00, 12.61it/s]


start processing Ch.47


100%|██████████| 5000/5000 [06:31<00:00, 12.77it/s]


start processing Ch.48


 82%|████████▏ | 4124/5000 [05:19<01:02, 14.09it/s]